In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import pandas as pd
import os
from datasets import Dataset
from sklearn.model_selection import KFold

In [3]:
!git clone https://github.com/Marcos-Med/IC.git

Cloning into 'IC'...
remote: Enumerating objects: 21634, done.
remote: Total 21634 (delta 0), reused 0 (delta 0), pack-reused 21634 (from 1)
Receiving objects: 100% (21634/21634), 21.46 MiB | 13.07 MiB/s, done.
Resolving deltas: 100% (6685/6685), done.
Updating files: 100% (21608/21608), done.


In [6]:
def load_corpus_fakeBR():
    base_path = "IC/fakebr/size_normalized_texts"

    fake_news = []
    true_news = []
    for root, dirs, files in os.walk(base_path + "/fake"):
        for file in files:
            if file.endswith('.txt'):
                with open(os.path.join(root, file), 'r', encoding='utf-8') as f:
                    text = f.read()
                    fake_news.append([text, 0])

    for root, dirs, files in os.walk(base_path + "/true"):
        for file in files:
            if file.endswith('.txt'):
                with open(os.path.join(root, file), 'r', encoding='utf-8') as f:
                    text = f.read()
                    true_news.append([text, 1])


    all_news = fake_news + true_news
    return pd.DataFrame(all_news, columns=['text', 'label'])

In [7]:
fake_br = load_corpus_fakeBR()
fake_br

,text,label
0,PROPINA 247: PF chega até blogueiro que mamava...,0
1,PGR recebe ofício pedindo a prisão de Lula por...,0
2,PF investiga ex-ministro petista suspeito de d...,0
3,Repórter da Folha ironiza atuação de Sérgio Mo...,0
4,"A senhorinha que calou a esquerda: ""Homem nu c...",0
...,...,...
7195,Suspeita de matar filha em Itapetininga coloca...,1
7196,Whindersson Nunes e outros: as 5 estrelas do Y...,1
7197,"'Não me envolvo em política', diz brasileiro q...",1
7198,Jovem morto pela PM após empinar moto tinha fo...,1


In [8]:
text = fake_br['text']
labels = fake_br['label']

In [9]:
model_name = "neuralmind/bert-base-portuguese-cased"

In [10]:
tokenizer = BertTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

In [11]:
def tokenize(data):
  return tokenizer(data['text'], padding='max_length', truncation=True, max_length=512)

In [12]:
dataset = Dataset.from_pandas(fake_br)
tokenized_dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/7200 [00:00<?, ? examples/s]

In [13]:
tokenized_dataset = tokenized_dataset.remove_columns(['text'])
tokenized_dataset.set_format("torch")

In [14]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [15]:
Kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [16]:
os.environ["WANDB_DISABLED"] = "true"

In [17]:
import numpy as np
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Cálculo das métricas
    accuracy = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions, average='binary')  # Para classificação binária
    recall = recall_score(labels, predictions, average='binary')  # Para classificação binária
    f1 = f1_score(labels, predictions, average='binary')  # Para classificação binária
    macro_f1 = f1_score(labels, predictions, average='macro')

    # Retornar as métricas calculadas
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "macro_f1": macro_f1
    }

In [18]:
results = []

In [19]:
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2) #Classificação binária

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
for train_index, test_index, in Kf.split(tokenized_dataset):
    train_dataset = tokenized_dataset.select(train_index.tolist())
    test_dataset = tokenized_dataset.select(test_index.tolist())

    # Paramêtros para realizar Fine Tuning no BERTimbau
    training_args = TrainingArguments(
        output_dir="./results",
        eval_strategy="epoch",
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=3,
        weight_decay=0.01,
        logging_dir="./logs",
        save_strategy="epoch"
        )
    #Objeto de treino
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics
    )

    #Treina o modelo
    trainer.train()

    #Avaliação do modelo BERTimbau
    metrics = trainer.evaluate()
    results.append(metrics)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Macro F1
1,0.159000,0.041972,0.990278,0.991667,0.988920,0.990291,0.990278
2,0.034000,0.035261,0.993750,0.994452,0.993075,0.993763,0.993750
3,0.007500,0.047761,0.992361,0.997203,0.987535,0.992345,0.992361


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Macro F1
1,0.062900,0.010413,0.997222,0.995714,0.998567,0.997139,0.997220
2,0.031200,0.004076,0.999306,1.000000,0.998567,0.999283,0.999305
3,0.004100,0.002598,0.999306,1.000000,0.998567,0.999283,0.999305


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Macro F1
1,0.048600,0.010932,0.997222,1.000000,0.994580,0.997283,0.997221
2,0.021400,0.002300,0.999306,1.000000,0.998645,0.999322,0.999305
3,0.000000,0.000405,1.000000,1.000000,1.000000,1.000000,1.000000


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Macro F1
1,0.046700,0.024337,0.996528,0.994505,0.998621,0.996559,0.996527
2,0.037800,0.022219,0.997222,1.000000,0.994483,0.997234,0.997222
3,0.011500,0.001086,0.999306,0.998623,1.000000,0.999311,0.999306


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Macro F1
1,0.043500,0.012537,0.997917,0.998603,0.997211,0.997906,0.997917
2,0.025100,0.005875,0.999306,1.000000,0.998605,0.999302,0.999306
3,0.004700,0.006363,0.999306,1.000000,0.998605,0.999302,0.999306


In [21]:
model.save_pretrained("save_model")
tokenizer.save_pretrained("save_tokenizer")

('save_tokenizer/tokenizer_config.json',
 'save_tokenizer/special_tokens_map.json',
 'save_tokenizer/vocab.txt',
 'save_tokenizer/added_tokens.json')

In [22]:
results

[{'eval_loss': 0.04776116460561752,
  'eval_accuracy': 0.9923611111111111,
  'eval_precision': 0.9972027972027973,
  'eval_recall': 0.9875346260387812,
  'eval_f1': 0.9923451635351427,
  'eval_macro_f1': 0.9923610779560675,
  'eval_runtime': 40.7907,
  'eval_samples_per_second': 35.302,
  'eval_steps_per_second': 4.413,
  'epoch': 3.0},
 {'eval_loss': 0.00259842025116086,
  'eval_accuracy': 0.9993055555555556,
  'eval_precision': 1.0,
  'eval_recall': 0.998567335243553,
  'eval_f1': 0.9992831541218637,
  'eval_macro_f1': 0.9993048767242315,
  'eval_runtime': 39.9239,
  'eval_samples_per_second': 36.069,
  'eval_steps_per_second': 4.509,
  'epoch': 3.0},
 {'eval_loss': 0.0004049500566907227,
  'eval_accuracy': 1.0,
  'eval_precision': 1.0,
  'eval_recall': 1.0,
  'eval_f1': 1.0,
  'eval_macro_f1': 1.0,
  'eval_runtime': 41.927,
  'eval_samples_per_second': 34.345,
  'eval_steps_per_second': 4.293,
  'epoch': 3.0},
 {'eval_loss': 0.0010864586802199483,
  'eval_accuracy': 0.99930555555555

In [23]:
mean_metrics = {
    "accuracy": np.mean([metrics["eval_accuracy"] for metrics in results]),
    "precision": np.mean([metrics["eval_precision"] for metrics in results]),
    "recall": np.mean([metrics["eval_recall"] for metrics in results]),
    "f1": np.mean([metrics["eval_f1"] for metrics in results]),
    "macro_f1": np.mean([metrics["eval_macro_f1"] for metrics in results])
}

In [24]:

# Exibir as métricas médias
print("Métricas médias após K-fold cross-validation:")
print(mean_metrics)

# Salvar em CSV
df_metrics = pd.DataFrame(results)
df_metrics.to_csv("metrics_kfold.csv", index=False)

df = pd.DataFrame([mean_metrics])
df.to_csv("mean_kfold.csv", index=False)

Métricas médias após K-fold cross-validation:
{'accuracy': 0.9980555555555556, 'precision': 0.9991650773468954, 'recall': 0.9969414522285728, 'f1': 0.9980482602149696, 'macro_f1': 0.9980554017712004}
